Packages

In [1]:
import torch
import numpy as np
import h5py
import os
import random
os.chdir(os.path.split(os.getcwd())[0])

Read data

In [2]:
with h5py.File(".\\dataset\\unitData.h5", 'r') as f:
  d = list(f["solEvalPoints"])
  # IDs of nodes used as trunk network inputs
  nodeID = list(map(int, d[0]))
  # [x y t] of y points (trunk inputs).
  # matrix with rows [x y t]
  solEvalPoint = torch.tensor(np.concatenate([d[1][:, None], d[2][:, None], d[3][:, None]], axis = 1))
  # DeepONet output targets (G(u)(y)) for all initial conditions
  # matrix with 'initial condition' rows and 'y point' columns
  solutionValue = torch.tensor(f["solutionValues"]).T
del d
print(len(nodeID))
print(solEvalPoint.shape)
print(solutionValue.shape)

1000
torch.Size([1000, 3])
torch.Size([1000, 1000])


C:\Users\kaoid\AppData\Local\Temp\ipykernel_2012\665812767.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:248.)
  solutionValue = torch.tensor(f["solutionValues"]).T


Data parameters

In [3]:
# number of sensors is equal to number of cases
nCase = solutionValue.shape[0]
nSensor = solutionValue.shape[0]
# value of concentrated load as initial condition
load = torch.tensor(-1.21 * 0.02).to(torch.float32)
evalAmount = solEvalPoint.shape[0] # number of y points (trunk inputs)
nSample = torch.numel(solutionValue) # amount of samples in dataset
print(f"Dataset contains {nSample} samples.")

Dataset contains 1000000 samples.


Organize data

In [4]:
class acoustDataset(torch.utils.data.Dataset):
    """Dataset subclass to organize data"""
    def __init__(self, evalPoint: torch.tensor, solVal: torch.tensor) -> None:
        super(acoustDataset, self).__init__()
        # repeat y coordinates for trunk input
        # self.trunkInput = torch.tile(evalPoint, (nSample // nCase, 1))
        self.trunkInput = evalPoint
        # target for model output
        self.target = torch.flatten(solVal)
        # load value at specific sensor. 0 everywhere else
        self.branchInput = torch.zeros((nSample, nSensor), dtype = torch.float32)
    
    def __getitem__(self, idx):
        """Overwrite method to fetch sample"""
        branchInput = torch.zeros(nSensor, dtype = torch.float32)
        branchInput[idx - (idx // nCase) * nCase] = load
        return branchInput, self.trunkInput[idx - (idx // evalAmount) * nCase, :], self.target[idx]
    
    def __len__(self):
        """Overwrite method to return size of dataset"""
        return torch.numel(self.target)

dataloader = torch.utils.data.DataLoader(acoustDataset(solEvalPoint, solutionValue),
    batch_size = 64, shuffle = True, num_workers = 0
)

In [54]:
branchBatch, trunkBatch, targetBatch = next(iter(dataloader))
print(f"""Batch shapes:
  branch network inputs: {branchBatch.shape}
  trunk network inputs: {trunkBatch.shape}
  model output targets: {targetBatch.shape}""")

Batch shapes:
  branch network inputs: torch.Size([64, 1000])
  trunk network inputs: torch.Size([64, 3])
  model output targets: torch.Size([64])
0 64


ML parameters

In [ ]:
branchDepth = 7 # number of layers in branch network (MLP architecture)
branchWidth = 100 # number of neurons in layers of branch network (MLP architecture)
trunkDepth = 7 # number of layers in trunk network (MLP architecture)
trunkWidth = 100 # number of neurons in layers of trunk network (MLP architecture)

Setup model

Loss and optimizer

Train